# 样本迁移学习实现指纹定位及baseline实验

In [1]:
import numpy as np
from sklearn import tree
from sklearn.preprocessing import scale  # 使用scikit-learn进行数据预处理
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression 
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import f1_score
from sklearn import preprocessing
import pandas as pd
from sklearn.datasets import load_iris
from collections import Counter
#import xgboost as xgb
import heapq as hp

/Users/amhu/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [2]:
training_data = pd.read_csv("/Users/amhu/OneDrive/In文件/trainingData.csv",engine='python',header = 0)
train_x = scale(np.asarray(training_data.iloc[:,0:520])) #归一化
train_y = np.asarray(training_data["BUILDINGID"].map(str) + training_data["FLOOR"].map(str))  
train_y = np.asarray(pd.get_dummies(train_y)) #进行one-hot  

test_dataset = pd.read_csv("/Users/amhu/OneDrive/In文件/validationData.csv",engine='python',header = 0)
test_x = scale(np.asarray(test_dataset.iloc[:,0:520]))
test_y = np.asarray(test_dataset["BUILDINGID"].map(str) + test_dataset["FLOOR"].map(str))
test_y = np.asarray(pd.get_dummies(test_y)) #进行one-hot  

/Users/amhu/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by the scale function.
  warnings.warn(msg, DataConversionWarning)


In [3]:
train_train_x,train_test_x,train_train_y,train_test_y = train_test_split(train_x,train_y,test_size = 0.3)
train_y_index = np.argmax(train_y,1)
test_y_index = np.argmax(test_y,1)
train_train_y_index = np.argmax(train_train_y,1)
train_test_y_index = np.argmax(train_test_y,1)

In [7]:
train_y[:10]

array([[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]], dtype=uint8)

In [6]:
test_train_x,test_test_x,test_train_y,test_test_y = train_test_split(test_x,test_y,test_size = 0.3)
test_train_y_index = np.argmax(test_train_y,1)
test_test_y_index = np.argmax(test_test_y,1)

In [11]:
test_train_y.shape

(777, 13)

1.baseline-------------------random_forest----------------------------------

In [11]:
# 标签是one-hot形式
# test 和 train 是不同批数据
clf_RF = RandomForestClassifier()
clf_RF.fit(train_x,train_y)
predicted_RF = clf_RF.predict(test_x)
acc = accuracy_score(predicted_RF,test_y)
print('random_forest准确率是{}'.format(acc))

random_forest准确率是0.48694869486948694


In [16]:
# 标签是类别形式
# test 和 train 是不同批数据
clf_RF2 = RandomForestClassifier()
clf_RF2.fit(train_x,train_y_index)
predicted_RF2 = clf_RF2.predict(test_x)
acc = accuracy_score(predicted_RF2,test_y_index)
print('random_forest准确率是{}'.format(acc))

random_forest准确率是0.693969396939694


In [17]:
# 标签是 类别 形式
# test 和 train 是同批数据
clf_RF3 = RandomForestClassifier()
clf_RF3.fit(train_train_x,train_train_y_index)
predicted_RF3 = clf_RF3.predict(train_test_x)
acc = accuracy_score(predicted_RF3,train_test_y_index)
print('准确率是{}'.format(acc))

准确率是0.9839518555667001


In [18]:
# 标签是 one_hot 形式
# test 和 train 是同批数据
clf_RF4 = RandomForestClassifier()
clf_RF4.fit(train_train_x,train_train_y)
predicted_RF4 = clf_RF4.predict(train_test_x)
acc = accuracy_score(predicted_RF4,train_test_y)
print('准确率是{}'.format(acc))

准确率是0.9577064526914075


In [29]:
# 标签是类别
# test 和 train 都是同批test数据
clf_RF5 = RandomForestClassifier()
clf_RF5.fit(test_train_x,test_train_y_index)
predicted_RF5 = clf_RF5.predict(test_test_x)
acc = accuracy_score(predicted_RF5,test_test_y_index)
acc_f1_score = f1_score(predicted_RF5,test_test_y_index,average='micro')
print('准确率为{},f1_score为{}'.format(acc,acc_f1_score))

准确率为0.811377245508982,f1_score为0.811377245508982


In [23]:
sum_train_x = np.concatenate((train_train_x,test_train_x),0)
sum_train_y = np.concatenate((train_train_y_index,test_train_y_index),0)

In [31]:
# 标签是类别
# trian 是联合数据
clf_RF7 = RandomForestClassifier()
clf_RF7.fit(sum_train_x,sum_train_y)
predicted_RF7 = clf_RF7.predict(test_test_x)
acc = accuracy_score(predicted_RF7,test_test_y_index)
acc_f1_score = f1_score(predicted_RF7,test_test_y_index,average='micro')
print('准确率为{},f1_score为{}'.format(acc,acc_f1_score))

准确率为0.8502994011976048,f1_score为0.8502994011976048


2.baseline-------------------Logistic----------------------------------

In [33]:
# 标签是one-hot
# test 和 train 是不同批数据
clf_LR = LogisticRegression(multi_class='multinomial',solver = 'lbfgs')
clf_LR.fit(train_x,train_y_index)
predicted_LR = clf_LR.predict(test_x)
acc = accuracy_score(predicted_LR,test_y_index)
print('Logistic准确率是{}'.format(acc))

Logistic准确率是0.6885688568856886


In [35]:
# 标签是类别
# test 和 train 是同批数据
clf_LR2 = LogisticRegression(multi_class='multinomial',solver = 'lbfgs')
clf_LR2.fit(train_train_x,train_train_y_index)
predicted_LR2 = clf_LR2.predict(train_test_x)
acc = accuracy_score(predicted_LR2,train_test_y_index)
print('准确率是{}'.format(acc))

准确率是0.9384821130056837


3.baseline-------------------GBDT----------------------------------

In [42]:
# 标签是one-hot
# test 和 train 是不同批数据
clf = GradientBoostingClassifier(n_estimators=50, learning_rate=1.0,max_depth=1, random_state=0).fit(train_x,train_y_index)
predicted_GB = clf.predict(test_x)
acc = accuracy_score(predicted_GB,test_y_index)
print('GBDT准确率是{}'.format(acc))

GBDT准确率是0.42034203420342037


In [43]:
# 标签是one-hot
# test 和 train 是不同批数据
clf_GB2 = GradientBoostingClassifier(n_estimators=50, learning_rate=1.0,
...     max_depth=1, random_state=0).fit(train_train_x,train_train_y_index)
predicted_GB2 = clf.predict(train_test_x)
acc = accuracy_score(predicted_GB2,train_test_y_index)
print('GBDT准确率是{}'.format(acc))

random_forest准确率是0.7271815446339017


4.baseline-------------------KNN----------------------------------

In [46]:
neigh = KNeighborsClassifier(n_neighbors = 3)
neigh.fit(train_x,train_y_index)
predicted_KNN = neigh.predict(test_x)
acc = accuracy_score(predicted_KNN,test_y_index)
print('KNN准确率是{}'.format(acc))

KNN准确率是0.6498649864986499


In [47]:
neigh2 = KNeighborsClassifier(n_neighbors = 3)
neigh2.fit(train_train_x,train_train_y_index)
predicted_KNN2 = neigh2.predict(train_test_x)
acc = accuracy_score(predicted_KNN2,train_test_y_index)
print('KNN准确率是{}'.format(acc))

KNN准确率是0.948345035105316


In [70]:
min(test_y_index)

0

In [80]:
xg_train = xgb.DMatrix(train_x, label = train_y_index)
xg_test  = xgb.DMatrix(test_x,label = test_y_index)
param = {'max_depth': 2, 'eta': 0.1, 'silent': 1, 'objective': 'multi:softmax'}
param['nthread'] = 4
param['eval_metric'] = 'merror'
param['num_class'] = 13
watchlist = [ (xg_train,'train'), (xg_test, 'test') ]
num_round = 25

In [81]:
bst = xgb.train(param, xg_train, num_round, watchlist )

[0]	train-merror:0.312183	test-merror:0.49955
[1]	train-merror:0.246577	test-merror:0.473447
[2]	train-merror:0.223153	test-merror:0.457246
[3]	train-merror:0.214576	test-merror:0.439244
[4]	train-merror:0.192406	test-merror:0.423042
[5]	train-merror:0.187842	test-merror:0.40324
[6]	train-merror:0.181271	test-merror:0.391539
[7]	train-merror:0.171741	test-merror:0.382538
[8]	train-merror:0.165822	test-merror:0.372637
[9]	train-merror:0.169133	test-merror:0.372637
[10]	train-merror:0.155139	test-merror:0.359136
[11]	train-merror:0.151477	test-merror:0.361836
[12]	train-merror:0.146662	test-merror:0.348335
[13]	train-merror:0.140743	test-merror:0.340234
[14]	train-merror:0.13337	test-merror:0.342934
[15]	train-merror:0.129759	test-merror:0.340234
[16]	train-merror:0.127552	test-merror:0.327633
[17]	train-merror:0.122335	test-merror:0.323132
[18]	train-merror:0.115113	test-merror:0.313231
[19]	train-merror:0.1111	test-merror:0.312331
[20]	train-merror:0.108793	test-merror:0.311431
[21]	tr

In [84]:
pred = bst.predict( xg_test ); 
print ('predicting, classification error=%f' % (sum( int(pred[i]) != test_y_index[i] for i in range(len(test_y_index))) / float(len(test_y_index)) ))

predicting, classification error=0.306031


5.experience-------------------样本迁移----------------------------------

In [50]:
def tradaboost(trans_S, trans_A, label_S, label_A, test, test_label,N,M):
    
    # A是辅助域，S是源域
    # A+S 是训练集
    # 输入的label都是one-hot形式
    trans_data = np.concatenate((trans_A, trans_S), axis=0)
    trans_label = np.concatenate((label_A, label_S), axis=0)
    all_data_AStest = np.concatenate((trans_data, test), axis=0)
     
#     row_A = trans_A.shape[0]
#     row_S = trans_S.shape[0]
#     row_T = test.shape[0]

    row_A = len(trans_A)
    row_S = len(trans_S)
    row_T = len(test_label)
    classifer_weights = np.zeros(N)
    
    # 初始化权重.weights_AS是只有辅助域和源域
    weights_A = np.ones([row_A, 1]) / row_A
    weights_S = np.ones([row_S, 1]) / row_S
    weights_AS = np.concatenate((weights_A, weights_S), axis=0)
    print('params initial finished.')
    
    # bata 是固定值，在源域用
    # bata_T 是变化值，在辅助域用
    bata = 1 / (1 + np.sqrt(2 * np.log(row_A / N)))
    bata_T = np.zeros([1, N])
    
    # 输出的label 是类别.result_lable是所有的源域、辅助域、test的输出，predict是测试集输出
    result_label_AStest = np.ones([row_A + row_S + row_T, N])
    predict = np.zeros([row_T])
    
    for i in range(N): 
        P = calculate_P(weights_AS)
        # 分类器的最终输出是 A S test 的分类结果,类别
        print(P.reshape(len(P),1).shape)
        
        result_label_AStest[:, i] = train_classify(trans_data, trans_label, all_data_AStest, P)
        print( 'result_sample', result_label_AStest[:, i][1:10], row_A, row_S, i, result_label_AStest.shape)
        
        #计算在源域的错误率
        # 计算在test 的错误率
        #error_rate = 1-f1_score(test_label,result_label_AStest[row_A + row_S:, i],average='micro')
        error_rate = 1-accuracy_score(test_label,result_label_AStest[row_A + row_S:, i])
        classifer_weights [i] =0.5*np.log((1-error_rate)/error_rate)
        print('test_Error rate:', error_rate)
        
        if error_rate > 0.5:
            error_rate = 0.5
        if error_rate == 0:
            N = i
            break  # 防止过拟合
            # error_rate = 0.001
        bata_T[0, i] = error_rate / (1 - error_rate)
        
        #通过在辅助域的正确与否，调整辅域样本权重
        for j in range(row_A):   
             weights_AS[j] = weights_AS[j] * np.power(bata, compare(result_label_AStest[j, i] , label_A[j]))
                
        # 过在源域的正确与否，调整源域样本权重
        for j in range(row_S):
            weights_AS[row_A + j] = weights_AS[row_A + j] * np.power(bata_T[0, i],-compare(result_label_AStest[row_A + j, i] ,int(label_S[j])))
    
    # 对N个分类器进行综合，得到最终的分类结果  
    y_predic = get_perdict(result_label_AStest[row_A + row_S:, :],classifer_weights,N)
    acc = accuracy_score(y_predic,test_label) 
    sample = obtain_sample(weights_AS,trans_data,trans_label,M)
    
    return acc,weights_AS,sample


def get_perdict(label_result,classifer_weights,N):
    sample_num = label_result.shape[0] #样本数量
    enc = preprocessing.OneHotEncoder(n_values=[13])  # one-hot 编码
    sum = np.zeros((sample_num,13))  #第一个维度是样本数，第二个维度是类别数，所有分类器的加权和
    for i in range(N): #将每个分类器的预测结果进行加权和       
        label_1 = label_result[:,i].reshape(sample_num,1) #对应的是每个分类器的所有样本预测值，维度结构是（样本数，1）
        label_onehot = enc.fit_transform(label_1)
        sum = sum+ label_onehot*classifer_weights[i]
    y_predict = np.argmax(sum,1)
    return y_predict

def train_classify(trans_data, trans_label, all_data_AStest, P):
#     clf = tree.DecisionTreeClassifier(criterion="gini", max_features="log2", splitter="random")
#     clf.fit(trans_data, trans_label,P.reshape(len(P)))
#     return clf.predict(all_data_AStest)
    clf_LR6 = RandomForestClassifier()
    #clf_LR6.fit(trans_data,trans_label,P.reshape(len(P)))
    clf_LR6.fit(trans_data,trans_label)

    return clf_LR6.predict(all_data_AStest)
   

def obtain_sample(weights_AS,trans_data,trans_label,M):
    sample = []
    index_max = weights_AS.reshape(len(weights_AS)).argsort()[-M:][::-1] 
    all = np.concatenate((trans_data,trans_label.reshape(len(trans_label),1)),1)
    for i in index_max:
        sample.append(all[int(i)])
    return sample

def calculate_P(weights,):
    total = np.sum(weights)
    return np.asarray(weights / total, order='C')        

def compare(a,b):
    if a==b:
        return 1
    else:
        return 0

In [66]:
trans_S = train_train_x #test_x[:350] 
label_S = train_train_y_index #np.argmax(test_y[:350],1)
 
trans_A = test_train_x #train_x[:1000]  
label_A = test_train_y_index #np.argmax(train_y[:1000],1)

test = test_test_x #test_x[351:]
test_label = test_test_y_index #np.argmax(test_y[351:],1)

In [67]:
acc,weights_AS,sample = tradaboost(trans_S, trans_A, label_S, label_A, test, test_label,10,1000)

params initial finished.
(14732, 1)
result_sample [10.  9.  2.  2.  0.  2.  1.  7.  3.] 777 13955 0 (15066, 10)
test_Error rate: 0.14371257485029942
(14732, 1)
result_sample [10.  9.  2.  2.  0.  2.  1.  7.  3.] 777 13955 1 (15066, 10)
test_Error rate: 0.16167664670658688
(14732, 1)
result_sample [10.  9.  2.  2.  0.  2.  1.  7.  3.] 777 13955 2 (15066, 10)
test_Error rate: 0.16766467065868262
(14732, 1)
result_sample [10.  9.  2.  2.  0.  2.  1.  7.  3.] 777 13955 3 (15066, 10)
test_Error rate: 0.16167664670658688
(14732, 1)
result_sample [10.  9.  2.  2.  0.  2.  1.  7.  3.] 777 13955 4 (15066, 10)
test_Error rate: 0.13772455089820357
(14732, 1)
result_sample [10.  9.  2.  2.  0.  2.  1.  7.  3.] 777 13955 5 (15066, 10)
test_Error rate: 0.1467065868263473
(14732, 1)
result_sample [10.  9.  2.  2.  0.  2.  1.  7.  3.] 777 13955 6 (15066, 10)
test_Error rate: 0.1467065868263473
(14732, 1)
result_sample [10.  9.  2.  2.  0.  2.  1.  7.  3.] 777 13955 7 (15066, 10)
test_Error rate: 0.167

In [68]:
acc

0.8832335329341318